<a href="https://colab.research.google.com/github/VIKAS-BUDHANI/Fake-News-Detaction/blob/main/Fake_news_detaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Dataset Downloaded from Kaggle
* Data set link  - https://www.kaggle.com/datasets/pnkjgpt/fake-news-datase
* Data set contains two file - Train and Test data



In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Data Preprocessing


In [ ]:
training_data = pd.read_csv("/content/drive/MyDrive/ML and DA projects/ML Project/Fake News Dectation/train.csv")

In [ ]:
training_data.head() 

,index,title,text,subject,date,class,Unnamed: 6
0,0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,"Aug 2, 2017",Fake,NaN
1,1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,"Oct 4, 2016",Fake,NaN
2,2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,"February 13, 2016",Fake,NaN
3,3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,"November 9, 2016",Real,NaN
4,4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,"May 26, 2017",Real,NaN


In [ ]:
training_data.shape
# There are 40,000 rows and 7 columns

(40000, 7)

In [ ]:
training_data.info()
#define the nulls and datatype of the data set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       40000 non-null  int64 
 1   title       40000 non-null  object
 2   text        40000 non-null  object
 3   subject     40000 non-null  object
 4   date        40000 non-null  object
 5   class       40000 non-null  object
 6   Unnamed: 6  1 non-null      object
dtypes: int64(1), object(6)
memory usage: 2.1+ MB


In [ ]:
training_data.isnull().sum()
# looks line there is no null vlaues prensent in data set except unnamed column

index             0
title             0
text              0
subject           0
date              0
class             0
Unnamed: 6    39999
dtype: int64

In [ ]:
training_data["class"].value_counts()
#there is only one row which having the wrong data

Fake                20886
Real                19113
February 5, 2017        1
Name: class, dtype: int64

In [ ]:
training_data[training_data["class"]=="February 5, 2017"]
# the wrong data row we can delete the row and also we have rectify the rows by shif the data 

,index,title,text,subject,date,class,Unnamed: 6
504,504,Even Mitch McConnell Is Beginning To Turn On ...,Is Impeachment Next? (VIDEO),Donald Trump s Muslim ban is proving to be a l...,News,"February 5, 2017",Fake


In [ ]:
# I have droped the column which having wrong data entered
training_data=training_data.drop(training_data[training_data["index"]==504].index)


In [ ]:
training_data.shape

(39999, 7)

In [ ]:
training_data["class"].value_counts()

Fake    20886
Real    19113
Name: class, dtype: int64

In [ ]:
training_data.info()
# We have unnamed:6 0 non null we can drop the column cause it us not nessesary for our model we can also drop the index  and date column it is not making sense

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39999 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       39999 non-null  int64 
 1   title       39999 non-null  object
 2   text        39999 non-null  object
 3   subject     39999 non-null  object
 4   date        39999 non-null  object
 5   class       39999 non-null  object
 6   Unnamed: 6  0 non-null      object
dtypes: int64(1), object(6)
memory usage: 2.4+ MB


In [ ]:
# droping rhe unnesssary columns
training_data.drop(columns=["index","Unnamed: 6","date"],inplace =True)

In [ ]:
training_data.head()

,title,text,subject,class
0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,Fake
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,Fake
2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,Fake
3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,Real
4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,Real


In [ ]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39999 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    39999 non-null  object
 1   text     39999 non-null  object
 2   subject  39999 non-null  object
 3   class    39999 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [ ]:
# printing the 1 row data 
training_data.iloc[0].value_counts()

PRESIDENT TRUMP Explains New “America First” RAISE Act…No More Welfare For New Immigrants, Migrants…Favors English Speaking Immigrants…Protects Jobs For Minorities, US Workers From Being Replaced And MORE! [VIDEO]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
# checking subject wise news
training_data["subject"].value_counts()

politicsNews       10075
worldnews           9038
News                8027
politics            6126
left-news           3966
Government News     1393
Middle-east          691
US_News              683
Name: subject, dtype: int64

In [ ]:
# percentage of fake and real news

training_data["class"].value_counts()/training_data.shape[0]


Fake    0.522163
Real    0.477837
Name: class, dtype: float64

# Text Processing 

In [ ]:
training_data.head()

,title,text,subject,class
0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,Fake
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,Fake
2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,Fake
3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,Real
4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,Real


In [ ]:
def text_preprocessing(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W"," ",text) 
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)    
  return text

* **text.lower() :** This line converts all characters in the text to lowercase. This is done to standardize the text and avoid having multiple representations of the same word with different capitalization.

* **re.sub('\[.*?\]', '', text) :** This line removes all text that is enclosed within square brackets. This is useful for removing text that might contain irrelevant information, such as URLs, metadata, or footnotes.

* **re.sub("\\W"," ",text) :** This line replaces all non-alphanumeric characters in the text with whitespace. This is useful for removing punctuation, symbols, and other special characters that do not contribute to the meaning of the text.

* **re.sub('https?://\S+|www\.\S+', '', text) :** This line removes all URLs or web addresses in the text. This is useful for removing hyperlinks or web references that might not be relevant to the text content.

* **re.sub('<.*?>+', '', text):** This line removes all HTML tags from the text. This is useful for removing any formatting or styling elements that might be present in the text, such as font styles, links, or images.

* **re.sub('[%s]' % re.escape(string.punctuation), '', text) :** This line removes all punctuation from the text. This is useful for removing any punctuation marks that might interfere with text analysis, such as periods, commas, or brackets.

* **re.sub('\n', '', text) :** This line removes all line breaks or newlines from the text. This is useful for ensuring that the text is represented as a continuous sequence of characters.

* **re.sub('\w*\d\w*', '', text) :** This line removes all alphanumeric sequences that contain digits. This is useful for removing numbers and any other alphanumeric sequences that might not be relevant to the text content.


In [ ]:
training_data["text"]= training_data["text"].apply(text_preprocessing)

In [ ]:
training_data.head()

,title,text,subject,class
0,PRESIDENT TRUMP Explains New “America First” R...,that s what we re talking about another campa...,politics,Fake
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,how is it that sean hannity is the only media ...,politics,Fake
2,Cruz Humiliated By Moderator After Lie About ...,almost immediately after learning that longtim...,News,Fake
3,"Russia revels in Trump victory, looks to sanct...",moscow reuters for all their mutual praise...,politicsNews,Real
4,Trump's bid to open U.S. monuments to developm...,washington reuters the trump administratio...,politicsNews,Real


In [ ]:
training_data["class"]=training_data["class"].replace("Fake",0)
training_data["class"]=training_data["class"].replace("Real",1)
# Fake = 0
# real = 1

In [ ]:
training_data.head()

,title,text,subject,class
0,PRESIDENT TRUMP Explains New “America First” R...,that s what we re talking about another campa...,politics,0
1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,how is it that sean hannity is the only media ...,politics,0
2,Cruz Humiliated By Moderator After Lie About ...,almost immediately after learning that longtim...,News,0
3,"Russia revels in Trump victory, looks to sanct...",moscow reuters for all their mutual praise...,politicsNews,1
4,Trump's bid to open U.S. monuments to developm...,washington reuters the trump administratio...,politicsNews,1


In [ ]:
# Spliting the data into depended and independed data
x = training_data["text"]
y = training_data["class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Vectorization 

NLP vectorization converts the text data into numberical format

Term Frequency-Inverse Document Frequency (TF-IDF): TF-IDF is a refinement of the BoW algorithm that takes into account the relative importance of each word in the document corpus. It assigns a weight to each word based on how frequently it appears in a document and how rare it is across the corpus.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
pred_lr=lr.predict(xv_test)

In [ ]:
# accurancy of logistic Regression model
lr.score(xv_test, y_test)


0.987

# Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc= RandomForestClassifier(random_state=0)
rfc.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rfc = rfc.predict(xv_test)

In [ ]:
# accurancy of random forest model
rfc.score(xv_test,y_test)

0.9907

# SVM (Support Vecotr Machine) Model

In [ ]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC()


In [ ]:
svm_model.fit(xv_train,y_train)

LinearSVC()

In [ ]:
pred_svm=svm_model.predict(xv_test)

In [ ]:
# Accurance of SVM model
svm_model.score(xv_test,y_test)

0.9945

# Decision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = dt.predict(xv_test)

In [ ]:
dt.score(xv_test,y_test)

0.9957

In [ ]:
# Best MOdel
print("Accuracy liner regression : ", lr.score(xv_test, y_test))
print("Accuracy of random forest Model : ",rfc.score(xv_test,y_test))
print("Accuracy of SVM modek : ",svm_model.score(xv_test,y_test))
print("Accuracy of Decission Tree Mdoel : ",dt.score(xv_test,y_test) )

Accuracy liner regression :  0.987
Accuracy of random forest Model :  0.9907
Accuracy of SVM modek :  0.9945
Accuracy of Decission Tree Mdoel :  0.9957


# Importing testing data


In [ ]:
testing_data = pd.read_csv("/content/drive/MyDrive/ML and DA projects/ML Project/Fake News Dectation/train.csv")

In [ ]:
testing_data.head()

,index,title,text,subject,date,class,Unnamed: 6
0,0,PRESIDENT TRUMP Explains New “America First” R...,That s what we re talking about! Another campa...,politics,"Aug 2, 2017",Fake,NaN
1,1,TERMINALLY ILL FORMER MISS WI: “Until my last ...,How is it that Sean Hannity is the only media ...,politics,"Oct 4, 2016",Fake,NaN
2,2,Cruz Humiliated By Moderator After Lie About ...,Almost immediately after learning that longtim...,News,"February 13, 2016",Fake,NaN
3,3,"Russia revels in Trump victory, looks to sanct...",MOSCOW (Reuters) - For all their mutual praise...,politicsNews,"November 9, 2016",Real,NaN
4,4,Trump's bid to open U.S. monuments to developm...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,"May 26, 2017",Real,NaN


# I have Used Logistic regression for Manual Testing the data

In [ ]:
#Manual testing of News 
# Data can be used from testing dataset
def prediction(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(text_preprocessing) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_lr = lr.predict(new_xv_test)
    

    return print(prediction(pred_lr[0]))

In [ ]:
#  Use Testing data for checking the model
news = str(input())
manual_testing(news)

Almost immediately after learning that longtim...	
Fake News
